<h2>Cálculo da solução ótima</h2>
<br>
<br>
Através da curva ABC, iremos trabalhar com <strong>20% dos produtos</strong> da farmácia que representam cerca de <strong>80% da soma total anual</strong> dos remédios da farmácia.<br>
Dito isso, nossa amostra serão os produtos das linhas <strong>1 a 562.</strong> Com a quantidade de linhas de 2118. 


In [59]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pulp

In [1]:
#Bibliotecas
import pandas as pd
from pulp import *

In [2]:
#Entrando com a tabela
dados = pd.read_csv("relatorio.csv").head(562)
dados.head(10)

,PRODUTO_ID,NCM,EAN,DESCRICAO,APRESENTACAO,QUANTIDADE,TOTAL_BRUTO,TOTAL_LIQUIDO,PROD_SALDO,VALOR_ULT_COMPRA,Unnamed: 10,SOMA TOTAL,Produtos A,Produtos B,Produtos C
0,21081,30049079.0,7.891320e+12,CITRATO DE SILDENAFILA 50MG COMP,50mg cx 8 comp rev,100,12426.600190,6213.210075,4.0,4.105300,NaN,"$530,724.79",80.00%,15.00%,5.00%
1,8501,30049037.0,7.896710e+12,TORSILAX C/ 10 COMPR. NEO QUIM.,CX/ 30 COMPR.,886,10289.360200,5178.870199,-12.0,2.290889,NaN,NaN,NaN,NaN,NaN
2,17371,30049069.0,7.896640e+12,NEOSALDINA 30 DRAG,NaN,315,9293.980021,7436.830130,13.0,22.237499,NaN,NaN,Valor Limite,NaN,NaN
3,8912,30039049.0,7.896520e+12,CIMELIDE C/12 COMPR.(NIMESULIDA),NIMESULIDA CIMED,497,7971.510023,4008.039969,71.0,2.524600,NaN,NaN,"$424,579.83","$79,608.72","$26,536.24"
4,19551,30049039.0,7.891060e+12,DORFLEX,300+35+50 mg comp x 300 emb. mult,51,7811.520081,4922.370056,1.0,117.339996,NaN,NaN,NaN,NaN,NaN
5,13028,30049079.0,7.891720e+12,MELOXICAM 15 MG.C/10 COMP. MERCK,GENERICO MERCK,134,4978.990086,2510.040016,5.0,6.455000,NaN,Linha inicial,Linha Limite,NaN,NaN
6,16367,30049067.0,7.896710e+12,CITRATO DE SILDENAFILA,50mg cx 4 comp,196,4653.140087,3124.250049,49.0,3.320000,NaN,1,562,1200,2119
7,18412,30049099.0,7.896470e+12,MULTIGRIP,400+4+4mg 2bl x 10 cap,198,4108.739998,2059.699992,33.0,4.290000,NaN,NaN,562,1200,2119
8,16276,30049099.0,7.898160e+12,FLORENT,100MG CX 12 CAP,104,3844.139931,1964.170044,6.0,12.188750,NaN,NaN,NaN,NaN,NaN
9,18969,30049077.0,7.899100e+12,BETRICORT,creme bg 30g,145,3826.599955,1915.829967,17.0,5.310000,NaN,Quantidade Linhas,2118,NaN,NaN


In [3]:
limite = 562

In [4]:
#Esse vetor possui todos os valores (TOTAL_BRUTO) dos produtos pertencentes a classe A
#Os valores desse vetor posteriormente vão ser trabalhados
valorClasseA = []
for index,item in enumerate(dados.TOTAL_BRUTO):
    if (index < limite):
        valorClasseA.insert(index,float(item))
    else:
        break

In [5]:
#Esse vetor possui todos os valores (QUANTIDADE) dos produtos pertencentes a classe A
#Os valores desse vetor posteriormente vão ser trabalhados
quantidadeClasseA = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        quantidadeClasseA.insert(index,item)
    else:
        break

In [6]:
mediaVenda = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        media = quantidadeClasseA[index] / 12
        mediaVenda.insert(index,media)
    else:
        break

In [7]:
lucro = []
for index,item in enumerate(dados.TOTAL_LIQUIDO):
    if (index < limite):
        lucro.insert(index, float(item)/quantidadeClasseA[index])
    else:
        break

In [8]:
desvioPadrao = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        desvioPadrao.insert(index, mediaVenda[index] * 0.1)
    else:
        break

In [9]:
vendaMinima = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        vendaMinima.insert(index, (mediaVenda[index] - desvioPadrao[index]))
    else:
        break

In [10]:
vendaMaxima = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        vendaMaxima.insert(index, (quantidadeClasseA[index] + (12*desvioPadrao[index])))
    else:
        break

In [11]:
giroEstoque = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        giroEstoque.insert(index, float(item)/(float(item) + float(dados.PROD_SALDO[index])))
    else:
        break

In [12]:
custoUnitario = []
for index,item in enumerate(dados.TOTAL_LIQUIDO):
    if (index < limite):
        custoUnitario.insert(index, (valorClasseA[index]-float(item))/quantidadeClasseA[index])
    else:
        break

In [13]:
capital = 0
for index,item in enumerate(dados.TOTAL_LIQUIDO):
    if (index < limite):
        capital += valorClasseA[index]-float(item)
    else:
        break

In [14]:
espacoTotal = 0
for item in quantidadeClasseA:
    espacoTotal += item

In [16]:
#Cria o problema de Programacao Linear para Maximizacao
prob = LpProblem("Maximização de lucros - Farmácia", LpMaximize)

In [17]:
#Cria as variáveis: Elas representam a quantidade inteira de cada produto que sera comprada para um ano
xi = []
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        xi.insert(index, LpVariable("q"+str(index),0,None,"Integer",None))
    else:
        break

In [18]:
#Cria a funcao objetivo, composta pelo produto do lucro unitario de um produto e sua quantidade (variavel)
obj = None
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        obj += lucro[index]*xi[index]
    else:
        break

In [19]:
#Adiciona a função objetivo ao problema
prob += obj

In [20]:
#Restrições - Venda Minima
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        prob += xi[index] >= vendaMinima[index]
    else:
        break

In [21]:
#Restrições - Venda Maxima
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        prob += xi[index] <= vendaMaxima[index]
    else:
        break

In [22]:
#Restrições - Espaço
obj = None
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        obj += xi[index]
    else:
        break

In [23]:
#Adiciona a restricao de espaco ao problema
prob+= obj <= espacoTotal

In [24]:
#Soluciona o problema e retorna um inteiro que indica o status da resolucao
prob.solve()

1

In [25]:
#Imprimindo o status da resolucao
print("Status: ", LpStatus[prob.status])

Status:  Optimal


In [26]:
#Imprimindo os valores das variáveis
for cont in prob.variables():
    print(cont.name, " = ", cont.varValue)

q0  =  110.0
q1  =  974.0
q10  =  293.0
q100  =  41.0
q101  =  34.0
q102  =  39.0
q103  =  332.0
q104  =  38.0
q105  =  15.0
q106  =  53.0
q107  =  31.0
q108  =  18.0
q109  =  7.0
q11  =  468.0
q110  =  15.0
q111  =  49.0
q112  =  47.0
q113  =  30.0
q114  =  15.0
q115  =  6.0
q116  =  78.0
q117  =  25.0
q118  =  18.0
q119  =  80.0
q12  =  130.0
q120  =  20.0
q121  =  7.0
q122  =  11.0
q123  =  7.0
q124  =  23.0
q125  =  39.0
q126  =  6.0
q127  =  7.0
q128  =  23.0
q129  =  25.0
q13  =  4.0
q130  =  246.0
q131  =  17.0
q132  =  457.0
q133  =  26.0
q134  =  16.0
q135  =  52.0
q136  =  45.0
q137  =  9.0
q138  =  42.0
q139  =  7.0
q14  =  205.0
q140  =  8.0
q141  =  9.0
q142  =  117.0
q143  =  26.0
q144  =  49.0
q145  =  8.0
q146  =  61.0
q147  =  18.0
q148  =  13.0
q149  =  13.0
q15  =  336.0
q150  =  20.0
q151  =  6.0
q152  =  41.0
q153  =  69.0
q154  =  45.0
q155  =  36.0
q156  =  9.0
q157  =  36.0
q158  =  6.0
q159  =  28.0
q16  =  31.0
q160  =  101.0
q161  =  42.0
q162  =  50.0
q163  

In [84]:
custo = 0
#Calculo Custo Total
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        custo += custoUnitario[index]*prob.variables()[index].varValue
    else:
        break
print(custo)

463560.08720016875


In [85]:
espaco = 0
#Calculo Espaco Total
for index,item in enumerate(dados.QUANTIDADE):
    if (index < limite):
        espaco += prob.variables()[index].varValue
    else:
        break
print(espaco)

21031.0


In [86]:
#Imprimindo o valor do custo total
print("Lucro total: ", value(prob.objective))

Lucro total:  301484.8555397317
